<a href="https://colab.research.google.com/github/mayurijoshi106/mayuri_INFO5731_spring2020/blob/master/INFO5731_Assignment_TWO_MJ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Two**

In this assignment, you will try to gather text data from open data source via web scraping or API. After that you need to clean the text data and syntactic analysis of the data.

# **Question 1**

(40 points). Write a python program to collect text data from **either of the following sources** and save the data into a **csv file**:

(1) Collect all the customer reviews of the product [2019 Dell labtop](https://www.amazon.com/Dell-Inspiron-5000-5570-Laptop/dp/B07N49F51N/ref=sr_1_11?crid=1IJ7UWF2F4GHH&keywords=dell%2Bxps%2B15&qid=1580173569&sprefix=dell%2Caps%2C181&sr=8-11&th=1) on amazon.

(2) Collect the top 100 User Reviews of the film [Joker](https://www.imdb.com/title/tt7286456/reviews?ref_=tt_urv) from IMDB.

(3) Collect the abstracts of the top 100 research papers by using the query [natural language processing](https://citeseerx.ist.psu.edu/search?q=natural+language+processing&submit.x=0&submit.y=0&sort=rlv&t=doc) from CiteSeerX.

(4) Collect the top 100 tweets by using hashtag ["#wuhancoronovirus"](https://twitter.com/hashtag/wuhancoronovirus) from Twitter. 


In [0]:
#!pip install scrapy
#!scrapy startproject amz_reviews_scraping
#!scrapy genspider amz_reviews_scraping/amz_reviews_scraping/spiders/amz_reviews https://www.amazon.com/Dell-Inspiron-5000-5570-Laptop/product-reviews/B07N49F51N/ref=cm_cr_getr_d_paging_btm_next_7?ie=UTF8&reviewerType=all_reviews&pageNumber=1
!scrapy runspider amz_reviews_scraping/amz_reviews_scraping/spiders/amz_reviews.py -o amazonreviews.csv

2020-02-27 20:13:48 [scrapy.utils.log] INFO: Scrapy 1.8.0 started (bot: scrapybot)
2020-02-27 20:13:48 [scrapy.utils.log] INFO: Versions: lxml 4.2.6.0, libxml2 2.9.8, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 19.10.0, Python 3.6.9 (default, Nov  7 2019, 10:44:02) - [GCC 8.3.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1d  10 Sep 2019), cryptography 2.8, Platform Linux-4.14.137+-x86_64-with-Ubuntu-18.04-bionic
2020-02-27 20:13:48 [scrapy.crawler] INFO: Overridden settings: {'CONCURRENT_REQUESTS': 1, 'DOWNLOAD_DELAY': 5, 'FEED_FORMAT': 'csv', 'FEED_URI': 'amazonreviews.csv', 'SPIDER_LOADER_WARN_ONLY': True}
2020-02-27 20:13:48 [scrapy.extensions.telnet] INFO: Telnet Password: dc2069a4430eadd5
2020-02-27 20:13:48 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2020-02-27 20:13:48 

# **Question 2**

(30 points). Write a python program to **clean the text data** you collected above and save the data in a new column in the csv file. The data cleaning steps include:

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the [stopwords list](https://gist.github.com/sebleier/554280).

(4) Lowercase all texts

(5) Stemming. 

(6) Lemmatization.

In [0]:
import pandas as pd
import nltk
import string
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import re

amzreview_df = pd.read_csv('/content/amazonreviews.csv')
amzreview_df['CleanData'] = amzreview_df['Reviews']
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
regex = re.compile('[^a-zA-Z\s]')

#1.1 - 1.2 Removes numbers, punctuation and special characters.
def remove_noise_special_numbers(text):
    return re.sub(regex, '', text)

#1.3 Removes stop words
def remove_stopwords(text):
  words = " ".join([word for word in text.split() if word not in stopwords.words('english')])
  return words

#1.5 Stemming
def stemming(text):
  stem = " ".join([stemmer.stem(i) for i in text.split()])
  return stem

#1.6 Lemmitization
def lemmatization(text):
  lem = " ".join([lemmatizer.lemmatize(i) for i in text.split()])
  return lem

amzreview_df['CleanData'] = amzreview_df['CleanData'].apply(lambda x: remove_noise_special_numbers(x))
#1.4 Lowercase all texts
amzreview_df['CleanData'] = amzreview_df['CleanData'].apply(lambda x: x.lower())
amzreview_df['CleanData'] = amzreview_df['CleanData'].apply(lambda x: remove_stopwords(x))
amzreview_df['CleanData'] = amzreview_df['CleanData'].apply(lambda x: stemming(x))
amzreview_df['CleanData'] = amzreview_df['CleanData'].apply(lambda x: lemmatization(x))

amzreview_df.to_csv('/content/amazonreviews.csv')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# **Question 3**

(30 points). Write a python program to conduct **syntax and structure analysis** of the clean text you just saved above. The syntax and structure analysis includes: 

(1) Parts of Speech (POS) Tagging: Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) Constituency Parsing and Dependency Parsing: print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) Named Entity Recognition: Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [0]:
import spacy
import nltk
nltk.download('punkt')
import en_core_web_sm
from spacy import displacy
from collections import Counter
nltk.download('averaged_perceptron_tagger')

amzreview_df = pd.read_csv('/content/amazonreviews.csv')

all_sentence = ""
for row in amzreview_df.itertuples():
  sentence = row.CleanData + " "
  all_sentence = all_sentence + sentence

#3.1 Parts of Speech Tagging
nltk_pos_tagged = nltk.pos_tag(all_sentence.split())
pd.DataFrame(nltk_pos_tagged, columns=['Word', 'POS tag'])
word_counts = Counter(tag[0] for word,tag in nltk_pos_tagged)
print ('Total number of Nouns in the text:' + str(word_counts['N']))
print ('Total number of Verbs in the text:' + str(word_counts['V']))
print ('Total number of Adjectives in the text:' + str(word_counts['J']))
print ('Total number of Adverbs in the text:' + str(word_counts['R']))
print ('POS Tagging for all the words in the text:')
for x in nltk_pos_tagged:
  print(x)

#3.3 Named Entity Recognition
ner = en_core_web_sm.load()
nersen = ner(all_sentence)

all_count = Counter(ent.label_ for ent in nersen.ents)
name_count = all_count['PERSON']
date_count = all_count['DATE']
org_count = all_count['ORG']
loc_count = all_count['LOC']
prod_count = all_count['PRODUCT']

print('\n\n*********************************************\n\n')
print('Number of date entity:' + str(date_count))
print('Number of person name entity:' + str(name_count))
print('Number of organizations entity:' + str(org_count))
print('Number of locations entity:' + str(loc_count))
print('Number of product name entity:' + str(prod_count))

for ent in nersen.ents:
  if(ent.label_ == 'DATE' or ent.label_ == 'ORG' or ent.label_ == 'PRODUCT' or ent.label_ == 'LOC' or ent.label_ == 'PERSON'):
    print(ent.text, ent.label_)




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
Total number of Nouns in the text:176
Total number of Verbs in the text:54
Total number of Adjectives in the text:96
Total number of Adverbs in the text:31
POS Tagging for all the words in the text:
('give', 'VB')
('sister', 'NN')
('cook', 'NN')
('clean', 'JJ')
('take', 'VB')
('trash', 'NN')
('love', 'VB')
('facebook', 'NN')
('new', 'JJ')
('laptop', 'JJ')
('bought', 'VBD')
('laptop', 'JJ')
('spec', 'NN')
('laptop', 'JJ')
('gobb', 'NN')
('memori', 'NNS')
('enough', 'RB')
('hard', 'JJ')
('drive', 'NN')
('space', 'NN')
('million', 'CD')
('pictur', 'IN')
('hope', 'NN')
('like', 'IN')
('im', 'JJ')
('pretti', 'JJ')
('sure', 'JJ')
('love', 'NN')
('far', 'RB')
('week', 'NN')
('use', 'NN')
('love', 'V

**Write your explanations of the constituency parsing tree and dependency parsing tree here (Question 3-2):** 